# Prepare

In [2]:
import json
import re
import os, sys
import pickle

In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm

In [4]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [5]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
def clean_sentence(sent):
    sent = sent.strip()
    sent = re.sub('{laughing}|{clearing}|{singing}|{applauding}', '', sent)
    sent = re.sub('[(][(].*?[)][)]|-.*?-', '', sent)
    return sent

In [7]:
get_main_topic = lambda x: x.split(' > ')[0]

def load_data():
    file_list = [f_name for f_name in os.listdir('data') if f_name[-5:] == '.json']
    
    total_data = []
    for f_name in tqdm(file_list):
        with open('data/%s' %f_name, 'r') as f:
            data = json.loads(f.read())['document'][0]
            metadata = data['metadata']
            utterance = data['utterance']
            
            topic = get_main_topic(metadata['topic'])
            if topic[:4] == 'NWRW':
                continue

            last_speaker = None
            seg1 = seg2 = ''
            for u in utterance:
                if last_speaker is None:
                    last_speaker = u['speaker_id']
                    seg2 = u['form']
                elif last_speaker == u['speaker_id']:
                    seg2 += ' ' + u['original_form']
                else:
                    if seg1 and seg2:
                        total_data.append([f_name, topic, clean_sentence(seg1), clean_sentence(seg2)])
                    last_speaker = u['speaker_id']
                    seg1 = seg2
                    seg2 = u['original_form']
            if seg1 and seg2:
                total_data.append([f_name, topic, clean_sentence(seg1), clean_sentence(seg2)])
                
    return np.array(total_data)

In [8]:
device = torch.device("cuda:0")

In [9]:
bertmodel, vocab = get_pytorch_kobert_model()
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model
using cached model
using cached model


In [10]:
## Setting parameters
pre_max_len = 4096
max_len = 512
batch_size = 6
warmup_ratio = 0.1
num_epochs = 20
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [11]:
indices = np.indices([pre_max_len])[0]
def pad_transform(transform, sent1, sent2, max_len):
    tokens, valid_len, segs = transform([sent1, sent2])
    if valid_len < max_len:
        return tokens[:max_len], np.array((tokens != 1).sum(), dtype='int32'), segs[:max_len]
    
    idx2 = indices[segs == 1][:-1]
    idx1 = indices[:idx2[0]][1:-1]
    if len(idx1) < len(idx2) and (len(idx1) * 2 + 3) < max_len:
        idx2 = idx2[:(max_len - 3 - len(idx1))]
    elif len(idx2) < len(idx1) and (len(idx2) * 2 + 3) < max_len:
        idx1 = idx1[-(max_len - 3 - len(idx2)):]
    else:
        if len(idx1) < len(idx2):
            idx1, idx2 = idx1[-((max_len - 3) // 2):], idx2[:((max_len - 3) // 2 + 1)]
        else:
            idx1, idx2 = idx1[-((max_len - 3) // 2 + 1):], idx2[:((max_len - 3) // 2)]
    return (np.concatenate([[2], tokens[idx1], [3], tokens[idx2],[3]]), 
             np.array(max_len, dtype='int32'), 
             np.array([0] * (len(idx1) + 2) + [1] * (len(idx2) + 1)))

In [12]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx1, sent_idx2, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=pre_max_len, pad=pad, pair=pair)

        self.sentences = [pad_transform(transform, d[sent_idx1], d[sent_idx2], max_len) for d in tqdm(dataset)]
#         self.sentences = [transform([d[sent_idx1], d[sent_idx2]]) for d in tqdm(dataset)]
        self.labels = [np.int32(d[label_idx]) for d in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [13]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        ret = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        pooler = ret[1]
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

# Train

In [19]:
def row_to_list(row):
    return str(row[2]), str(row[3]), label_w2i[row[1]]

In [36]:
raw_dataset = load_data()
file_with_label = np.unique(raw_dataset[:, :2], axis=0)
_labels, _counts = np.unique(file_with_label[:, 1], return_counts=True)
label_counts = dict(zip(_labels, _counts))

100%|██████████| 2232/2232 [00:02<00:00, 1062.10it/s]


In [37]:
np.random.seed(0)
np.random.shuffle(file_with_label)

valid_list = {label: [] for label in label_counts}
test_list = {label: [] for label in label_counts}

for file in file_with_label:
    if len(valid_list[file[1]]) < label_counts[file[1]] * 0.1:
        valid_list[file[1]].append(file[0])
    elif len(test_list[file[1]]) < label_counts[file[1]] * 0.1:
        test_list[file[1]].append(file[0])
        
_valid_list = []
_test_list = []
for label in label_counts:
    _valid_list.extend(valid_list[label])
    _test_list.extend(test_list[label])
    
valid_list = _valid_list
test_list = _test_list

In [38]:
raw_train_dataset, raw_valid_dataset, raw_test_dataset = [], [], []
for row in raw_dataset:
    if row[0] in valid_list:
        raw_valid_dataset.append(row)
    elif row[0] in test_list:
        raw_test_dataset.append(row)
    else:
        raw_train_dataset.append(row)
raw_train_dataset, raw_valid_dataset, raw_test_dataset = np.array(raw_train_dataset), np.array(raw_valid_dataset), np.array(raw_test_dataset)

In [48]:
np.savez('raw_train_valid_test.npz', raw_train_dataset, raw_valid_dataset, raw_test_dataset)

In [16]:
npz_file = np.load('raw_train_valid_test.npz')
raw_train_dataset, raw_valid_dataset, raw_test_dataset = npz_file['arr_0'], npz_file['arr_1'], npz_file['arr_2']

In [ ]:
label_i2w = np.unique(raw_train_dataset[:, 1]).tolist()
label_w2i = {w: i for i, w in enumerate(label_i2w)}

In [17]:
label_w2i = pickle.load(open('ver_2.w2i', 'rb'))
label_i2w = {label_w2i[l]: l for l in label_w2i}

In [20]:
train_dataset = [row_to_list(r) for r in raw_train_dataset]
valid_dataset = [row_to_list(r) for r in raw_valid_dataset]
test_dataset = [row_to_list(r) for r in raw_test_dataset]

In [21]:
data_train = BERTDataset(train_dataset, 0, 1, 2, tok, max_len, True, True)
data_valid = BERTDataset(valid_dataset, 0, 1, 2, tok, max_len, True, True)
data_test = BERTDataset(test_dataset, 0, 1, 2, tok, max_len, True, True)

100%|██████████| 2277/2277 [00:04<00:00, 494.87it/s]


In [22]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
valid_dataloader = torch.utils.data.DataLoader(data_valid, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [42]:
model = BERTClassifier(bertmodel, dr_rate=0.3, num_classes=len(label_i2w)).to(device)

In [43]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
# loss_fn = nn.CrossEntropyLoss(weight=weight)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [44]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [45]:
pickle.dump(label_w2i, open('ver_2.w2i', 'wb'))
result = []

n_batch = len(train_dataloader)
for e in range(num_epochs):
    train_acc = 0.0
    valid_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader, file=sys.stdout)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            tqdm.write("epoch %02d batch id %04d/%d loss %f train acc %f" %(e+1, batch_id+1, n_batch, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    train_acc /= batch_id + 1
    print("epoch %02d train acc %f" %(e+1, train_acc))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in in enumerate(tqdm(valid_dataloader, file=sys.stdout)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        valid_acc += calc_accuracy(out, label)
    valid_acc /=  batch_id+1
    print("epoch %02d valid acc %f" %(e+1, valid_acc))
    torch.save(model, 'ver_2_%02d_%.04f_%.04f.model' %(e+1, train_acc, valid_acc))

epoch 01 batch id 0001/3105 loss 2.619715 train acc 0.000000
epoch 01 batch id 0201/3105 loss 2.834718 train acc 0.051410
epoch 01 batch id 0401/3105 loss 2.598113 train acc 0.071072
epoch 01 batch id 0601/3105 loss 2.834365 train acc 0.118691
epoch 01 batch id 0801/3105 loss 2.500948 train acc 0.146484
epoch 01 batch id 1001/3105 loss 1.784213 train acc 0.228605
epoch 01 batch id 1201/3105 loss 0.629869 train acc 0.300999
epoch 01 batch id 1401/3105 loss 0.396200 train acc 0.371996
epoch 01 batch id 1601/3105 loss 1.481052 train acc 0.422965
epoch 01 batch id 1801/3105 loss 2.382094 train acc 0.463909
epoch 01 batch id 2001/3105 loss 0.571073 train acc 0.487756
epoch 01 batch id 2201/3105 loss 0.207719 train acc 0.513403
epoch 01 batch id 2401/3105 loss 0.254676 train acc 0.539012
epoch 01 batch id 2601/3105 loss 0.121420 train acc 0.562476
epoch 01 batch id 2801/3105 loss 0.012414 train acc 0.581935
epoch 01 batch id 3001/3105 loss 0.046614 train acc 0.593524
100%|██████████| 3105/31

epoch 08 batch id 2001/3105 loss 0.008381 train acc 0.857488
epoch 08 batch id 2201/3105 loss 0.004854 train acc 0.856505
epoch 08 batch id 2401/3105 loss 0.930413 train acc 0.859989
epoch 08 batch id 2601/3105 loss 0.007730 train acc 0.862681
epoch 08 batch id 2801/3105 loss 0.002061 train acc 0.865822
epoch 08 batch id 3001/3105 loss 0.082560 train acc 0.862768
100%|██████████| 3105/3105 [12:24<00:00,  4.17it/s]
epoch 08 train acc 0.864895
epoch 08 valid acc 0.805216
epoch 09 batch id 0001/3105 loss 0.004603 train acc 1.000000
epoch 09 batch id 0201/3105 loss 0.940431 train acc 0.853234
epoch 09 batch id 0401/3105 loss 0.007532 train acc 0.853283
epoch 09 batch id 0601/3105 loss 0.006902 train acc 0.871048
epoch 09 batch id 0801/3105 loss 0.007539 train acc 0.882022
epoch 09 batch id 1001/3105 loss 0.003453 train acc 0.888278
epoch 09 batch id 1201/3105 loss 0.005687 train acc 0.889675
epoch 09 batch id 1401/3105 loss 0.002893 train acc 0.892577
epoch 09 batch id 1601/3105 loss 0.015

epoch 16 batch id 0401/3105 loss 0.001738 train acc 0.938487
epoch 16 batch id 0601/3105 loss 0.011580 train acc 0.947033
epoch 16 batch id 0801/3105 loss 0.008540 train acc 0.950479
epoch 16 batch id 1001/3105 loss 0.002242 train acc 0.953047
epoch 16 batch id 1201/3105 loss 0.005804 train acc 0.954760
epoch 16 batch id 1401/3105 loss 0.002861 train acc 0.954318
epoch 16 batch id 1601/3105 loss 0.005317 train acc 0.954091
epoch 16 batch id 1801/3105 loss 3.830405 train acc 0.954655
epoch 16 batch id 2001/3105 loss 0.006163 train acc 0.951608
epoch 16 batch id 2201/3105 loss 0.005914 train acc 0.951386
epoch 16 batch id 2401/3105 loss 0.004140 train acc 0.952103
epoch 16 batch id 2601/3105 loss 0.005171 train acc 0.953544
epoch 16 batch id 2801/3105 loss 0.002097 train acc 0.953945
epoch 16 batch id 3001/3105 loss 0.004724 train acc 0.952849
100%|██████████| 3105/3105 [12:15<00:00,  4.22it/s]
epoch 16 train acc 0.952979
epoch 16 valid acc 0.833333
epoch 17 batch id 0001/3105 loss 0.001

# Predict

In [14]:
from sklearn.metrics import recall_score, precision_score, f1_score

In [39]:
model = torch.load('ver_2_12_0.9233_0.8150.model')
label_w2i = pickle.load(open('ver_2.w2i', 'rb'))
label_i2w = {label_w2i[l]: l for l in label_w2i}

In [40]:
def get_pred(X):
    _, indices = torch.max(X, 1)
    return indices.cpu().numpy().tolist()

In [41]:
valid_acc = 0
test_pred = []
model.eval()
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader, file=sys.stdout)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)
    test_pred.append(get_pred(out))
test_pred = np.concatenate(test_pred)

100%|██████████| 380/380 [00:28<00:00, 13.38it/s]


In [42]:
test_true = np.array([d[2] for d in test_dataset])

In [43]:
test_acc = (test_pred == test_true).sum() / len(test_true)
test_precision = [precision_score(test_true, test_pred, labels=[i], average='macro') for i in label_i2w]
test_recall = [recall_score(test_true, test_pred, labels=[i], average='macro') for i in label_i2w]
test_f1 = [f1_score(test_true, test_pred, labels=[i], average='macro') for i in label_i2w]

In [44]:
print('| Label | Precision | Recall | F1 |')
print('|-------|-----------|--------|----|')
for i, (precision, recall, f1) in enumerate(zip(test_precision, test_recall, test_f1)):
    print('| %s | %.4f | %.4f | %.4f |' %(label_i2w[i], precision, recall, f1))
print('| %s | %.4f | %.4f | %.4f |' %(
    'Total', 
    precision_score(test_true, test_pred, average='macro'), 
    recall_score(test_true, test_pred, average='macro'), 
    f1_score(test_true, test_pred, average='macro'), 
))

print('Test Accuracy : %.4f' %test_acc)

| Label | Precision | Recall | F1 |
|-------|-----------|--------|----|
| 가족 | 0.6932 | 0.5755 | 0.6289 |
| 건강/다이어트 | 0.9109 | 0.8364 | 0.8720 |
| 계절/날씨 | 0.9516 | 0.8872 | 0.9183 |
| 꿈(목표) | 0.7079 | 0.6702 | 0.6885 |
| 먹거리 | 0.9084 | 0.9189 | 0.9136 |
| 반려동물 | 0.9286 | 0.9832 | 0.9551 |
| 방송/연예 | 0.8628 | 0.7471 | 0.8008 |
| 선물 | 0.9153 | 0.9153 | 0.9153 |
| 성격 | 0.8540 | 0.9360 | 0.8931 |
| 스포츠/레저 | 0.8544 | 0.8232 | 0.8385 |
| 아르바이트 | 0.9469 | 0.9224 | 0.9345 |
| 여행지(국내/해외) | 0.8495 | 0.9518 | 0.8977 |
| 연애/결혼 | 0.8367 | 0.8849 | 0.8601 |
| 영화 | 0.8636 | 0.9553 | 0.9072 |
| 회사/학교 | 0.7010 | 0.7606 | 0.7296 |
| Total | 0.8523 | 0.8512 | 0.8502 |
Test Accuracy : 0.8546


In [37]:
file_names = [
    "ver_2_01_0.6018_0.8097.model",
    "ver_2_02_0.8018_0.7486.model",
    "ver_2_03_0.7981_0.7910.model",
    "ver_2_04_0.8213_0.8064.model",
    "ver_2_05_0.8524_0.7979.model",
    "ver_2_06_0.8199_0.7979.model",
    "ver_2_07_0.8499_0.8146.model",
    "ver_2_08_0.8649_0.8052.model",
    "ver_2_09_0.8882_0.8215.model",
    "ver_2_10_0.8847_0.7869.model",
    "ver_2_11_0.9118_0.7661.model",
    "ver_2_12_0.9233_0.8150.model",
    "ver_2_13_0.9323_0.8146.model",
    "ver_2_14_0.9406_0.8113.model",
    "ver_2_15_0.9473_0.8346.model",
    "ver_2_16_0.9530_0.8333.model",
    "ver_2_17_0.9570_0.8276.model",
    "ver_2_18_0.9588_0.8240.model",
    "ver_2_19_0.9611_0.8260.model",
    "ver_2_20_0.9627_0.8248.model"
]

In [38]:
for f in file_names:
    model = torch.load(f)
    label_w2i = pickle.load(open('ver_2.w2i', 'rb'))
    label_i2w = {label_w2i[l]: l for l in label_w2i}
    test_pred = []
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader, file=sys.stdout)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_pred.append(get_pred(out))
    test_pred = np.concatenate(test_pred)
    test_true = np.array([d[2] for d in test_dataset])
    test_acc = (test_pred == test_true).sum() / len(test_true)
    print(test_acc)

100%|██████████| 380/380 [00:29<00:00, 12.89it/s]
0.829161176987264
100%|██████████| 380/380 [00:29<00:00, 12.87it/s]
0.7307861220904699
100%|██████████| 380/380 [00:29<00:00, 12.78it/s]
0.7870004391743523
100%|██████████| 380/380 [00:29<00:00, 12.73it/s]
0.8120333772507685
100%|██████████| 380/380 [00:29<00:00, 12.74it/s]
0.8173034694773825
100%|██████████| 380/380 [00:29<00:00, 12.71it/s]
0.7782169521299956
100%|██████████| 380/380 [00:30<00:00, 12.66it/s]
0.8230127360562143
100%|██████████| 380/380 [00:29<00:00, 12.72it/s]
0.8278436539306104
100%|██████████| 380/380 [00:30<00:00, 12.65it/s]
0.8322353974527887
100%|██████████| 380/380 [00:29<00:00, 12.71it/s]
0.7834870443566095
100%|██████████| 380/380 [00:29<00:00, 12.71it/s]
0.7566974088713219
100%|██████████| 380/380 [00:29<00:00, 12.93it/s]
0.8546332894158981
100%|██████████| 380/380 [00:28<00:00, 13.12it/s]
0.8375054896794027
100%|██████████| 380/380 [00:28<00:00, 13.12it/s]
0.8274044795783926
100%|██████████| 380/380 [00:29<00:

In [35]:
test_pred

array([13, 13, 13, ...,  6,  6,  6])

In [15]:
raw_data = [
    ('혹시 마블 좋아하세요?', '네. 최근에 스파이더맨 봤어요.'),
    ('점심 뭐 드셨어요?', '근처에서 마라탕 먹었어요.')
]
data = BERTDataset([d + (0,) for d in raw_data], 0, 1, 2, tok, max_len, True, False)
dataloader = DataLoader(data, batch_size=batch_size, num_workers=5)

100%|██████████| 2/2 [00:00<00:00, 91.12it/s]


In [16]:
model.eval()
outputs = []
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length = valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)
    outputs.append(out.cpu().detach().numpy())

result = np.concatenate(outputs)

/home/svclaw2000/Dialog-Classification-Using-KoBERT/venv_kcc/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
result

array([[-1.6374834 , -1.3215848 ,  0.12254745, -2.5353487 , -0.89550805,
        -1.4683422 ,  1.9166542 , -2.0298762 , -1.7553192 ,  1.1083449 ,
        -1.3344933 , -0.5012208 ,  0.7741338 ,  8.988237  , -0.67473793],
       [ 0.14410633,  0.48958924, -0.43005556, -1.36853   , 10.405009  ,
        -1.5924621 , -1.6080155 ,  0.31998512, -2.137295  , -0.46647248,
        -0.10372277,  2.2267296 , -1.6280773 , -1.0927978 , -1.1962178 ]],
      dtype=float32)

In [62]:
full_length = list(map(lambda x: len(x[2] + x[3]), raw_train_dataset))

In [51]:
temp = np.array(test_dataset)

In [52]:
temp[(test_pred == test_true) & (test_true == 0)]

array([['아~ 언니 어제 엄마한테 전화가 왔는데 &name&이가 내가 동생이 있거든. 남동생이 근데 걔가 고등학교 이 학년이야. 근데 얘가 공부를 너무 안 해. 너무 안 해가지고 내보고 공부를 시켜달라는 거야. 쫌 어떻게 쫌 전화를 해가지고 &name&이가 전화를 해가지구 쫌 공부를 시키라 이렇게 나한테 말을 하는 거야. 그게 난 너무 짜증나가지고 이제 전화를 하긴 했다. 했는데 얘가 영 이제 사춘기가 뒤늦게 왔는지 영 얘가 퉁명스럽고 내가 그래도 수능 선밴데 내 얘기를 안 들을려하고 자기가 학원대로 할려는 거야. 아~ 그래서 너무 어제 스트레스 받아가지고 어~ 친언니도 있거든. 나 그래서 친언니랑 이제 둘이서 완전 얘를 어뜨케 해야 되나 하고 이제 스 같이 토론을 했던 그런 기억이 있거든. 어제 그~ 언니도 혹시 동생 있어?',
        '너랑 나랑 완전 똑같은 거 같은데 나도 언니 한 명이 있고 남동생 한 명 있거든. 근데 딱 역시 다들 비슷한 거 같애. 요새 내 남동생도 내 남동생은 공부를 스 잘하는 편이야. 그래가지구 어~ 그래가지고 자기 혼자서 공부를 하는데 이건 또 나쁜 점이 이제 나를 무시를 해. 나도 그래두 얘랑 성적이 비슷하 비슷했 하긴 했었는데 얘가 교육 과정이 달라지면서 이제 수학 문제 들고 와가지구 나한테 어~ 문제 풀 때 오 분에서 십 분 정도는 걸릴 거 아니야. 근데 가져와가지고 모른 척하면서 물어보고 내가 모르면 이제 어~ 너는 이런 것도 모르냐 아~ 너무 멍청하다 이런 식으로 말을 하고 도망가니까 이제 언니랑 나랑 어~ 이제 걔가 문제를 물어보면 거의 무시하는 정도로  지나쳐. 그런 걸 보면 너 남동생이랑 친한 거 같은데 언니랑도 친해?',
        '0'],
       ['너랑 나랑 완전 똑같은 거 같은데 나도 언니 한 명이 있고 남동생 한 명 있거든. 근데 딱 역시 다들 비슷한 거 같애. 요새 내 남동생도 내 남동생은 공부를 스 잘하는 편이야. 그래가지구 어~ 그래가지고 자기 혼자서 공부를 하는데 이건 또 나

In [70]:
dict(zip(np.where((test_pred != test_true) & (test_true == 0))[0], [label_i2w[i] for i in test_pred[(test_pred != test_true) & (test_true == 0)]]))

{61: '성격',
 413: '회사/학교',
 414: '회사/학교',
 415: '회사/학교',
 480: '성격',
 482: '성격',
 483: '성격',
 541: '연애/결혼',
 542: '연애/결혼',
 557: '꿈(목표)',
 558: '꿈(목표)',
 559: '여행지(국내/해외)',
 561: '반려동물',
 852: '연애/결혼',
 1470: '연애/결혼',
 1471: '연애/결혼',
 1472: '연애/결혼',
 1473: '연애/결혼',
 1474: '연애/결혼',
 1475: '연애/결혼',
 1957: '선물',
 2096: '회사/학교',
 2097: '회사/학교',
 2098: '회사/학교',
 2099: '먹거리',
 2136: '회사/학교',
 2137: '회사/학교',
 2138: '회사/학교',
 2139: '연애/결혼',
 2144: '방송/연예',
 2145: '스포츠/레저',
 2234: '먹거리',
 2235: '회사/학교',
 2236: '먹거리',
 2237: '먹거리',
 2243: '먹거리',
 2247: '꿈(목표)',
 2248: '여행지(국내/해외)',
 2249: '여행지(국내/해외)',
 2251: '먹거리',
 2252: '먹거리',
 2253: '먹거리',
 2254: '먹거리',
 2255: '먹거리',
 2256: '먹거리'}

In [72]:
np.where((test_pred == test_true) & (test_true == 5))

(array([ 302,  303,  304,  305,  306,  307,  308,  612,  613,  614,  615,
         616,  617,  618,  619,  620,  621,  622,  623,  624,  625,  626,
         627,  642,  643,  644,  682,  683,  684,  686,  687,  688,  689,
         690, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022,
        1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1033, 1034,
        1035, 1036, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1178,
        1179, 1180, 1181, 1182, 1183, 1184, 1185, 1186, 1187, 1476, 1477,
        1478, 1479, 1480, 1481, 1482, 1483, 1484, 1508, 1509, 1510, 1511,
        1512, 1513, 1514, 1626, 1627, 1628, 1629, 1630, 1631, 1837, 1838,
        1839, 1840, 1841, 1842, 1843, 1844, 1845, 2023, 2024, 2025, 2026,
        2027, 2113, 2114, 2115, 2116, 2117, 2118]),)

In [74]:
num = 303
label_i2w[test_pred[num]], test_dataset[num]

('반려동물',
 ('아~ 네. 그 모더콜리가 굉장히 이케 크기가 약간 크드라구요. 그러고 활동적이고 그래서 굉장히 산책을 좋아한다고 얘기 들었그든요? 그래서 같이 있었으면 정말 재밌었을 것 같애요. 제가 보기에도 저와 같은 경우에는 강아지를 키웠는데 굉장히 작은 종류를 키웠어요. 대체적으로요. 집에서 그냥 키울 수 있는 종류로 키웠는데 음~ 그~ 강아지를 누가 줬느냐 하면 예전에 큰  할머니가 주셨어요. 그래서 그 큰할머니가 그 강아지를 줬는데 그 강아지 눈이 우리 큰할머니 눈 하고 똑같앴어요. 그래서 걔만 보지 잘못해서 다른 데 뭐 오줌을 싸거나 그랬을 때 야단을 치고 싶지만은 그 큰할머니 눈을 이제 마주치는 것 같애서 야단을 못 쳤던 경험이 있고 굉장히 뭐 이렇게 야단칠려 그러면 가만히 있어서 도대체가 야단칠 수 없을 정도로 굉장히 예뻤던 강아지였어요. 근데 그 강아지도 보통 십오 년이 넘으면은 치매 증상이 있드라구요. 그래서 치매 증상이 있는데도 불구하고 내가 굉장히 함부로 오줌 싸는 걸 싫어해서 그런지 다리를 끌면서까지 화장실에 가갖고서 누트라구요. 그게 너무너무 사랑스러워서 다들 뭐 안락사를 시키라 그랬그든. 예전에 그런데 차마 그러진 못하고 끝까지 제가 보살폈던 경험이 있습니다. 아~ 그러면은 그~ 보더콜리  이 외에는 강아지를 키운 적이 없으신가요?',
  '예. 보더콜리가 가장 오랜 기간 키웠던 견종이 보더콜리였고 어릴 때부터 뭐 믹스견도 있었고 푸들도 있었고 어~  잠시였지만 진돗개도 어렸을 때 잠깐 키웠었습니다. 그래서 어~ 사실  저로서는 뭐 믹스견이든 뭐 품질이  품질이 뭐 좋다고 하는 견종이든 사실 크게 차이는 없다고 생각을 합니다. 뭐 반려동물이 사실 인간 인간과에 어떤 외로움을 공유할 수 있는 존재 그런 존재라고 생각을 하고 뭐 일상에 작은 작은 소소한 행복 이런 도구라고 생각을 하기 때문에 뭐 믹스견이든 뭐 푸들이든 뭐 견종에 대해서  그렇게 상관하진 않는 편이었구요. 어~ 굳이 다른 얘기를 하자면  진돗개는 뭐 사실 품 너무  

In [94]:
length_count = [[0, 0] for _ in label_i2w]

for d in data_train:
    length_count[d[3]][0] += 1 if d[1] == 512 else 0
    length_count[d[3]][1] += 1
    
for d in data_test:
    length_count[d[3]][0] += 1 if d[1] == 512 else 0
    length_count[d[3]][1] += 1
    
for d in data_valid:
    length_count[d[3]][0] += 1 if d[1] == 512 else 0
    length_count[d[3]][1] += 1

In [101]:
for i, (l, c) in enumerate(length_count):
    print('%.3f' %(l/c), label_i2w[i], sep='\t')

0.471	가족
0.377	건강/다이어트
0.233	계절/날씨
0.416	꿈(목표)
0.248	먹거리
0.401	반려동물
0.214	방송/연예
0.411	선물
0.344	성격
0.297	스포츠/레저
0.370	아르바이트
0.233	여행지(국내/해외)
0.323	연애/결혼
0.226	영화
0.247	회사/학교


In [96]:
data_train[0][1] == 512

True